In [ ]:
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [ ]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_motif"

In [ ]:
# Load nbS1-O1 circuit with connectivity matrix
circuit = obi.Circuit(name="nbS1-O1",
                      path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"),
                      matrix_path=os.path.join(cmat_root, "nbS1-O1", "connectivity_matrix.h5"))
display(circuit.connectivity_matrix.matrix)

In [ ]:
# Set up pair motif neuron sets

hex_nset = "nbS1-HEX0"
layers = ["2", "3", "4", "5", "6"]
inh_subtypes = ["hex_O1PV", "hex_O1Sst"]

# # (1) E->I sets per layer and INH subtype
# neuron1_filter = [{"node_set": hex_nset, "synapse_class": "EXC", "layer": _lay} for _lay in layers for _inh in inh_subtypes]
# neuron2_filter = [{"node_set": [hex_nset, _inh], "synapse_class": "INH", "layer": _lay} for _lay in layers for _inh in inh_subtypes]

# conn_ff_filter = {"nsyn": {"gt": 0}}
# conn_fb_filter = {"nsyn": 0}  # No feedback connection

# pair_selection = {"count": 1, "method": "max_nsyn_ff"}  # Selection based on max. number of synapses

# virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "POm") if (_lay == "6" and _inh == "hex_O1Sst") else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers for _inh in inh_subtypes]
# output_path = os.path.join(output_root, "E2I")

# # (2) I->E sets per layer and INH subtype
# neuron1_filter = [{"node_set": [hex_nset, _inh],  "synapse_class": "INH", "layer": _lay} for _lay in layers for _inh in inh_subtypes]
# neuron2_filter = [{"node_set": hex_nset, "synapse_class": "EXC", "layer": _lay} for _lay in layers for _inh in inh_subtypes]

# conn_ff_filter = {"nsyn": {"gt": 0}}
# conn_fb_filter = {"nsyn": 0}  # No feedback connection

# pair_selection = {"count": 1, "method": "max_nsyn_ff"}  # Selection based on max. number of synapses

# virtual_sources_to_ignore = ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", )
# output_path = os.path.join(output_root, "I2E")

# (3) E<->I sets per layer and INH subtype
neuron1_filter = [{"node_set": hex_nset, "synapse_class": "EXC", "layer": _lay} for _lay in layers for _inh in inh_subtypes]
neuron2_filter = [{"node_set": [hex_nset, _inh],  "synapse_class": "INH", "layer": _lay} for _lay in layers for _inh in inh_subtypes]

conn_ff_filter = {"nsyn": {"gt": 0}}
conn_fb_filter = {"nsyn": {"gt": 0}}  # Reciprocal connection

pair_selection = {"count": 1, "method": "max_nsyn_all"}  # Selection based on max. number of synapses

virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "POm") if (_lay == "6" and _inh == "hex_O1Sst") else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers for _inh in inh_subtypes]
output_path = os.path.join(output_root, "ErcI")

node_set_list_op = "intersect"
motif_neuron_sets = obi.PairMotifNeuronSet(neuron1_filter=neuron1_filter, neuron2_filter=neuron2_filter, conn_ff_filter=conn_ff_filter, conn_fb_filter=conn_fb_filter, pair_selection=pair_selection, node_set_list_op=node_set_list_op)

names = [f"{os.path.split(output_path)[-1]}-{_inh}-{hex_nset}-L{_lay}" for _lay in layers for _inh in inh_subtypes]

In [ ]:
circuit_extractions_scan_config= obi.CircuitExtractionScanConfig(
                    initialize=obi.CircuitExtractionScanConfig.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    neuron_set=motif_neuron_sets,
)

coupled_scan = obi.CoupledScanGenerationTask(form=circuit_extractions_scan_config, output_root=output_path, coordinate_directory_option="ZERO_INDEX")

In [ ]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
# coupled_scan.coordinate_instances(display=True)

In [ ]:
coupled_scan.execute()
obi.run_tasks_for_generated_scan(coupled_scan)

In [ ]:
# Check circuits
import json
from bluepysnap import Circuit

c = circuit.sonata_circuit
nodes = c.nodes["S1nonbarrel_neurons"]
hex0_ids = nodes.ids("nbS1-HEX0")

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    with open(coupled_scan.output_root / str(_idx) / "id_mapping.json", "r") as f:
        id_map = json.load(f)
    parent_ids = id_map["S1nonbarrel_neurons"]["parent_id"]
    assert np.all(np.isin(parent_ids, hex0_ids))

    nodes = c.nodes['S1nonbarrel_neurons']
    edges = c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"]
    nids = np.hstack([nodes.ids({"synapse_class": "EXC"}), nodes.ids({"synapse_class": "INH"})])
    print(f"{names[_idx]:27s}>>> {nodes.size} neurons, {edges.size} (FF {len(edges.pathway_edges(source=nids[0], target=nids[1]))}, FB {len(edges.pathway_edges(source=nids[1], target=nids[0]))}) synapses, m-types: {nodes.get(nids, properties=['mtype']).to_numpy().flatten().tolist()}")

